In [1]:
!python -V

Python 3.10.11


In [2]:
import pickle

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import mlflow
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error

In [3]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi")

2023/06/17 19:41:51 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2023/06/17 19:41:51 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
2023/06/17 19:41:51 INFO mlflow.tracking.fluent: Experiment with name 'nyc-taxi' does not exist. Creating a new experiment.


<Experiment: artifact_location='/workspaces/machine-learning/ml-ops-zoomcamp/lesson-2/mlruns/1', creation_time=1687030911207, experiment_id='1', last_update_time=1687030911207, lifecycle_stage='active', name='nyc-taxi', tags={}>

In [4]:
# Let's put the logic into a function so that we don't need to do a lot of copy-pasting
def read_dataframe(filename):
    """Perform pre-processing on a dataframe."""
    df = pd.read_parquet(f'./data/{filename}')
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)

    df = df[df.trip_type == 2]

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

In [5]:
# This means that the model is wrong on average by almost 6 minutes
# Let's try a different model: Lasso
df_train = read_dataframe('green_tripdata_2022-01.parquet')
df_val = read_dataframe('green_tripdata_2022-02.parquet')

In [6]:
len(df_train), len(df_val)

(1310, 1438)

In [7]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [8]:
# Let's do the same thing for the validation data
categorical = ['PU_DO'] # ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

# Set up training data again
train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

# Set up validation data
val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [9]:
y_train = df_train['duration'].values
y_val = df_val['duration'].values

In [10]:
with mlflow.start_run():

    mlflow.set_tag("developer", "caitlin")
    mlflow.log_param("train-data-path", "green_tripdata_2022-01.parquet")
    mlflow.log_param("valid-data-path", "green_tripdata_2022-02.parquet")
    

    lr = LinearRegression()
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)

    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

8.10808873353748

In [12]:
with open('./models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [13]:
# Let's try Lasso
lr = Lasso(0.01)
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.783033934402808

In [14]:
# Lastly, let's try Ridge
lr = Ridge()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.68958099596436